In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>'''
)

In [2]:
import ipywidgets as widgets
import pandas

from bokeh.io import output_notebook, show
from bokeh.plotting import figure

from bokeh.models import CustomJS
from bokeh.models.sources import ColumnDataSource
from bokeh.layouts import column, row
from bokeh.models.widgets import DataTable, TableColumn, Button, Panel, Tabs, HTMLTemplateFormatter

import re

from IPython.display import IFrame, clear_output, Image
from GenDBScraper.PseudomonasDotComScraper import PseudomonasDotComScraper, pdc_query

from GenDBScraper.StringDBScraper import StringDBScraper, stringdb_query

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
def run_pdc(locus_tag):

    pdc = PseudomonasDotComScraper(query=pdc_query(strain='sbw25', feature=locus_tag))
    query_string = "__".join([pdc.query[0].strain, pdc.query[0].feature])
    pdc.connect()
    pdc.run_query()
    
    clear_output(wait=True)
    
    results = pdc.results[query_string]
    panels=list(results.keys())

    children = []
    for i,panel in enumerate(panels):
        #print(panel)
        df = results[panel]
        if df is None:
            continue
        if isinstance(df, pandas.DataFrame):    
            if df.empty:
                continue
            df = df.rename(str, axis='columns')
            columns = [TableColumn(field=c, title=c) for c in list(df.columns)]
            for col in columns:
                if col.title == 'url':
                    col.formatter=HTMLTemplateFormatter(template='<a href="<%= url %>"><%= value %></a>')
        elif isinstance(df, dict):
            continue
    #        columns = [TableColumn(field=c, title=c) for c in df.keys()]
        elif isinstance(df, list):
            continue
    #        columns=[TableColumn(field=c, title=c) for c in df]

        cds = ColumnDataSource(data=df)
        data_table = DataTable(source=cds, columns=columns)
        children.append(Panel(child=data_table, title=panel))

    tabs = Tabs(tabs=children)

    show(tabs)

In [5]:
def sbw25_okm():
    clear_output()
    frame = IFrame(src="https://openknowledgemaps.org/map/4aafb7d70516de0f56190d374bf398c8&embed=true", width=1000, height=1000)
    display(frame)
    

In [6]:

def feature_okm(locus_tag):
    okms = pandas.read_json("pflu_okm_urls_20190424.json", typ='series', orient='records')

    if locus_tag.upper() in okms.keys():
        frame = IFrame(okms[locus_tag.upper()]+"&embed=true", width=900, height=800)

    else:
        frame = widgets.Label("No maps found for {}".format(locus_tag))

    clear_output(wait=True)    
    display(frame)

In [7]:
def run_stdb(locus_tag):
    gene_sub_pattern = re.compile(r'([a-z](?=[0-9]))')
    gene=gene_sub_pattern.sub(r'\1_', locus_tag)
    
    stdb = StringDBScraper(query=stringdb_query(taxonId=216595, features=[gene]))

    stdb.connect()

    stdb.update_features()

    stdb_results = dict()

    stdb_results['Network Image'] = stdb.network_image()
    stdb_results['Network Interactions'] = stdb.network_interactions()
    stdb_results['Interaction Partners'] = stdb.interaction_partners(required_score=300)
    stdb_results['Functional Enrichments'] = stdb.functional_enrichments()
    stdb_results['Interaction Enrichments'] = stdb.interaction_enrichments()
  
    clear_output(wait=True)
    with open(stdb_results['Network Image'], 'rb') as fp:
        image_widget = widgets.Image(value=fp.read(), format='svg')

    tabs = []
    for key in stdb_results.keys():
        if key == 'Network Image':
            continue
        result = stdb_results[key]
        cds = ColumnDataSource(result)
        data_table = DataTable(source=cds,
                               columns=[TableColumn(field=c, title=c, width=80) for c in list(result.columns)],
                               fit_columns=False
                              )
        tabs.append(Panel(child=data_table, title=key))
        
    stdb_tabs = Tabs(tabs=tabs)
    display(image_widget)
    show(column(stdb_tabs, width=500))

# The pseudomonas-fluorescens SBW25 knowledge base 

In [8]:
lt = r'pflu0001'
locus_tag = lt.lower()

# Displaying data for locus tag {{lt.upper()}}

## Data from pseudomonas.com 

In [9]:
run_pdc(locus_tag)

## Data from string-db.org 

In [10]:
run_stdb(locus_tag)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xcf\x00\x00\x01\xcd\x08\x06\x00\x00\x00 \xdcI\xe…

## OpenKnowledgeMaps
Reference: OpenKnowldegeMaps.org - Overview of research on sbw25. Retrieved from https://openknowledgemaps.org/map/2b4a41a5661a99a129d60b6583444e67. 

In [11]:
sbw25_okm()

In [12]:
feature_okm(locus_tag)